### Install alfalfa-client from the development branch

Skip this step if you have done it once already.

In [ ]:
%%bash
pip uninstall alfalfa-client
Y

In [ ]:
!pip install git+https://github.com/NREL/alfalfa-client@81541ea52e600dc07f1d8eac553cd0cf14968313

### Import the libraries

In [ ]:
import alfalfa_client.alfalfa_client as ac
import alfalfa_client.historian as ah

import os, requests, datetime

In [ ]:

client = ac.AlfalfaClient()
historian = ah.Historian()


### Submit your testcase. 
Each time we submit a testcase, the function returns its ID. We will need it for other client calls.

For more information: [Alfalfa's Wiki](https://github.com/NREL/alfalfa/wiki/OpenStudio-Model-Setup)

We upload a test case based on the DOE Prototype Primary School building.

In [ ]:
zippath = "examples/primaryschool.zip"
ref = client.submit(os.path.join('..', zippath))

### Start a testcase
To start a simulation, we need its ID, a start and end time. Here, we use an external clock to control the simulation. We set the start time to 7am on January 3rd.

In [ ]:

start = datetime.datetime(2022, 1, 3, 7, 0, 0)
end = (start + datetime.timedelta(days=365)).strftime('%Y-%m-%d %H:%M:%S')
start = start.strftime('%Y-%m-%d %H:%M:%S')
inp = client.start(ref, start_datetime=start, end_datetime=end, external_clock=True)

### Get testcase information

We can request a list of all the simulation outputs

In [ ]:
sensors = client.get_outputs(ref)
sensors

The same goes for the simulation inputs

In [ ]:
points = client.get_inputs(ref)
points

### Override controls

We can update the control vector at each timestep. One timestep is one minute. 

The control vector, u, is a dict where the key is one of the strings reported by the `get_inputs` method and the value is the value we want to set this point to.

In this example, we raise the heating and cooling setpoints in the `Mult_Class_2_Pod_2` classroom, `mult_class_2_pod_2_zn_1_flr_1_zn_heating_temperature_setpoint` and `mult_class_2_pod_2_zn_1_flr_1_zn_cooling_temperature_setpoint` respectively, for 2 hours.

In [ ]:
steps = 120
htg = [25 for i in range(steps)]
clg = [i + 4 for i in htg]

controlledzone = 'mult_class_2_pod_2_zn_1_flr_1 zn'
controlledcomponent = 'mult_class_2_pod_2_zn_1_flr_1 zn'
htgvar = 'Heating_Temperature'
clgvar = 'Cooling_Temperature'

for h, c in zip(htg, clg):
    u = {
    f"{controlledzone.replace(' ', '_')}_{htgvar}_Setpoint" : h,
    f"{controlledzone.replace(' ', '_')}_{clgvar}_Setpoint" : c,

    }
    response = client.set_inputs(ref, u)
    client.advance(ref)
    res = client.get_outputs(ref)
    ctime = client.get_sim_time(ref)
    zonetemp = res[f"{controlledzone} Zone Air Temperature Sensor"]
    htgval = res[f"{controlledzone.replace(' ', '_')}_{htgvar}_Setpoint_Value"]
    clgval = res[f"{controlledzone.replace(' ', '_')}_{clgvar}_Setpoint_Value"]
    print(f"\tTime: {ctime}\n\tZone temperature: {zonetemp}\tHeating STP: {htgval}\n\tCooling STP: {clgval}")

And below, we access the VAV terminal's control node and change its temperature setpoint. In this case, we also need to start the ventilation system by setting the air flow setpoint in order to see a change in discharge air temperature.

In [ ]:
steps = 120
ctrl = [30 for i in range(steps)]

controlledzone = 'mult_class_2_pod_2_zn_1_flr_1 zn'
controlledcomponent = 'mult_class_2_pod_2_zn_1_flr_1 zn vav terminal outlet air node'
tvar = 'Temperature'
mvar = 'Air_Flow'

for c in ctrl:
    u = {
    f"{controlledcomponent.replace(' ', '_')}_{tvar}_Setpoint" : c,
    f"{controlledcomponent.replace(' ', '_')}_{mvar}_Setpoint" : 1

    }
    response = client.set_inputs(ref, u)
    client.advance(ref)
    res = client.get_outputs(ref)
    ctime = client.get_sim_time(ref)
    zonetemp = res[f"{controlledzone} Zone Air Temperature Sensor"]
    tval = res[f"{controlledcomponent.replace(' ', '_')}_{tvar}_Setpoint_Value"]
    tsen = res[f"{controlledcomponent} {tvar} Sensor"]
    print(f"\tTime: {ctime}\n\tZone temperature: {zonetemp}\tDischarge air STP: {tval}\n\tDischarge air temperature sensor: {tsen}")

Finally, to stop a testcase:

In [ ]:
client.stop(ref)